In [1]:
import pandas as pd
import psycopg2
import numpy as np

In [2]:
try: 
    conn=psycopg2.connect(
       host='sumar.larioja.gob.ar',
       user='postgres',
       password='sumarLR21',
       database='sumar'
       )
    
    print("conexion exitosa")
    cursor = conn.cursor()
    cursor.execute("SELECT version()")
    row=cursor.fetchone()
    print(row)
    
except Exception as ex:
    print(ex)

conexion exitosa
('PostgreSQL 9.0.1, compiled by Visual C++ build 1500, 32-bit',)


In [3]:
# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Consulta SQL para obtener la lista de esquemas
query = "SELECT schema_name FROM information_schema.schemata"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
esquemas = cursor.fetchall()

# Imprimir los esquemas
print("Lista de esquemas:")
for esquema in esquemas:
    print(esquema[0])

# Cerrar el cursor y la conexión
cursor.close()

Lista de esquemas:
pg_toast
pg_temp_1
pg_toast_temp_1
pg_catalog
information_schema
alta
calidad
capacitacion
cardiopatia
centro
contabilidad
expediente
facturacion
fichero
general
leche
mensajes
monitoreo
nacer
nomenclador
permisos
personal
planillas
pss
public
puco
sistema
tareas_divisionsoft
trazadoras
uad
laravel


In [4]:
# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Esquema específico que deseas consultar
esquema_especifico = "facturacion"

# Consulta SQL para obtener las tablas en un esquema específico
query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{esquema_especifico}'"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
tablas = cursor.fetchall()

# Imprimir las tablas
print(f"Tablas en el esquema '{esquema_especifico}':")
for tabla in tablas:
    print(tabla[0])

# Cerrar el cursor y la conexión
cursor.close()

Tablas en el esquema 'facturacion':
prueba
validacion_diagnostico_copy
controlnomenclador
efec_nom
nomenclador
lineacuidado
nomenclador1
nomenclador_backup
notadebito
nomenclador2
notificacion
periodo
prestacionesdebito
smiefectores
validacion_prestacion_minimas
validacion_diagnostico
prestacionesceb
credito
debito
servicio
motivo_d
anexo
validacion_prestacion
comprobante
factura
log_comprobante
log_factura
log_prestacion
nomenclador_detalle
nomenclador_old
prestacion
afiliados_baja
ajustarfacturas


In [5]:
# Consulta SQL para obtener los datos de una tabla en un esquema específico
query = """
		SELECT
			p.precio_prestacion,
			p.fecha_prestacion,
			p.monto_debito,
			c.cuie,
			c.periodo,
			f.estado,
			n.subgrupo,
			REPLACE(n.codigo, '-', '') AS codigo
FROM "facturacion"."prestacion" AS p
LEFT JOIN "facturacion"."comprobante" AS c ON p.id_comprobante = c.id_comprobante
LEFT JOIN "facturacion"."factura" AS f ON c.id_factura = f.id_factura
LEFT JOIN "facturacion"."nomenclador" AS n ON p.id_nomenclador = n.id_nomenclador
WHERE p.fecha_prestacion > '2023-01-01';
"""
# Crear un DataFrame a partir de la consulta
df_prestaciones = pd.read_sql_query(query, conn)

# Imprimir el DataFrame
print(df_prestaciones)

# Cerrar la conexión
cursor.close()
conn.close()

C:\Users\FRANCISCO\AppData\Local\Temp\ipykernel_7880\638041920.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_prestaciones = pd.read_sql_query(query, conn)


         precio_prestacion fecha_prestacion  monto_debito    cuie  periodo  \
0                     40.0       2023-09-13           0.0  L03944  2023/09   
1                    380.0       2023-09-06           0.0  L03911  2023/09   
2                    700.0       2023-09-08           0.0  L03888  2023/09   
3                    550.0       2023-09-25           0.0  L00684  2023/09   
4                    550.0       2023-09-13           0.0  L00727  2023/09   
...                    ...              ...           ...     ...      ...   
1006432              700.0       2023-09-21           0.0  L00692  2023/09   
1006433               90.0       2023-08-03           0.0  L00951  2023/08   
1006434               90.0       2023-08-18           0.0  L00001  2023/08   
1006435               90.0       2023-08-16           0.0  L03923  2023/08   
1006436               90.0       2023-08-22           0.0  L03877  2023/08   

        estado      subgrupo     codigo  
0            P       

In [6]:
df_prestaciones['periodo'] = pd.to_datetime(df_prestaciones['periodo'])

df_prestaciones['monto_debito'] = df_prestaciones['monto_debito'].astype(int)

df_prestaciones['fecha_prestacion'] = pd.to_datetime(df_prestaciones['fecha_prestacion'])

df_prestaciones['precio_prestacion'] = df_prestaciones['precio_prestacion'].astype(int)

C:\Users\FRANCISCO\AppData\Local\Temp\ipykernel_7880\3530792970.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_prestaciones['periodo'] = pd.to_datetime(df_prestaciones['periodo'])


In [7]:
df_prestaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006437 entries, 0 to 1006436
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   precio_prestacion  1006437 non-null  int32         
 1   fecha_prestacion   1006437 non-null  datetime64[ns]
 2   monto_debito       1006437 non-null  int32         
 3   cuie               1006437 non-null  object        
 4   periodo            1006437 non-null  datetime64[ns]
 5   estado             961135 non-null   object        
 6   subgrupo           1006437 non-null  object        
 7   codigo             1006437 non-null  object        
dtypes: datetime64[ns](2), int32(2), object(4)
memory usage: 53.7+ MB


In [8]:
df_prestaciones

,precio_prestacion,fecha_prestacion,monto_debito,cuie,periodo,estado,subgrupo,codigo
0,40,2023-09-13,0,L03944,2023-09-01,P,Taller,TAT013A98
1,380,2023-09-06,0,L03911,2023-09-01,P,Imagenología,IGR031
2,700,2023-09-08,0,L03888,2023-09-01,P,Consulta,CTC001A97
3,550,2023-09-25,0,L00684,2023-09-01,P,Consulta,CTC011A97
4,550,2023-09-13,0,L00727,2023-09-01,P,Consulta,CTC010A97
...,...,...,...,...,...,...,...,...
1006432,700,2023-09-21,0,L00692,2023-09-01,P,Consulta,CTC006W78
1006433,90,2023-08-03,0,L00951,2023-08-01,C,Inmunización,IMV013A98
1006434,90,2023-08-18,0,L00001,2023-08-01,P,Inmunización,IMV013A98
1006435,90,2023-08-16,0,L03923,2023-08-01,P,Inmunización,IMV013A98


In [9]:
df_prestaciones.to_csv('tablas_powerBI/df_prestaciones.csv', index=False)